In [1]:
import torch
import os
import torch.nn as nn
import numpy as np
from torch.utils.data import DataLoader
from network import CharacterClassifier
from tqdm import tqdm
from training_data import HASY
from PIL import Image
from matplotlib import pyplot as plt

hasy_train = HASY('HASY')
hasy_test = HASY('HASY', train=False)

print("Train data length: {0}".format(len(hasy_train.data)))
print("Test data length: {0}".format(len(hasy_test.data)))
print("Img Shape: {0}".format(hasy_train.data[0].shape))
print("Number of Labels: {0}".format(hasy_train.no_labels))

100%|██████████| 16992/16992 [00:00<00:00, 80154.59it/s]

Train data length: 1958
Test data length: 224
Img Shape: torch.Size([3, 36, 36])
Number of Labels: 15


In [3]:
from torchvision import models

train_loader = DataLoader(hasy_train, batch_size=16, shuffle=True)
test_loader = DataLoader(hasy_test, batch_size=16, shuffle=False)

def calc_accuracy(model):
    accuracies = []
    for idx, [x_test, y_test] in enumerate(test_loader):
        test_pred = model(x_test)
        accuracy = 100 * torch.mean((torch.argmax(test_pred, dim=1) == y_test).float())
        accuracies.append(accuracy)
    return np.mean(accuracies) 

def print_stats(lr, beta1, beta2, decay, accuracy):
    print("Learning rate: {0}".format(lr))
    print("Beta 1: {0}".format(beta1))
    print("Beta 2: {0}".format(beta2))
    print("Weight decay: {0}".format(decay))
    print("Accuracy: {0}".format(accuracy))

learning_rates = [0.01, 0.001, 0.0001]
betas1 = [0.8, 0.85, 0.9, 0.95]
betas2 = [0.9, 0.925, 0.95, 0.99]
weight_decays = [0, 0.01, 0.001, 0.0001]

best_lr = 0
best_beta1 = 0
best_beta2 = 0
best_decay = 0
best_accuracy = 0

for lr in learning_rates:
    for beta1 in betas1:
        for beta2 in betas2:
            for decay in weight_decays:
                model = models.densenet201(num_classes=15)
                optimizer = torch.optim.Adam(model.parameters(), lr=lr, betas=(beta1, beta2), weight_decay=decay)
                criterion = nn.CrossEntropyLoss()
                for step, [x_train, y_train] in enumerate(tqdm(train_loader)):
                    optimizer.zero_grad()
                    train_pred = model(x_train)
                    loss = criterion(train_pred, y_train)
                    loss.backward()
                    optimizer.step()
                accuracy = calc_accuracy(model)
                print_stats(lr, beta1, beta2, decay, accuracy)
                if accuracy > best_accuracy:
                    best_lr = lr
                    best_beta1 = beta1
                    best_beta2 = beta2
                    best_decay = decay
                    best_accuracy = accuracy
                    
print("Best hyperparameters:")
print_stats(best_lr, best_beta1, best_beta2, best_decay, best_accuracy)
                

100%|██████████| 123/123 [04:10<00:00,  1.70s/it]


Learning rate: 0.01
Beta 1: 0.8
Beta 2: 0.9
Weight decay: 0
Accuracy: 31.25


100%|██████████| 123/123 [03:55<00:00,  1.73s/it]


Learning rate: 0.01
Beta 1: 0.8
Beta 2: 0.9
Weight decay: 0.01
Accuracy: 29.464284896850586


100%|██████████| 123/123 [03:47<00:00,  1.71s/it]


...

Learning rate: 0.0001
Beta 1: 0.95
Beta 2: 0.99
Weight decay: 0.01
Accuracy: 33.92856979370117


100%|██████████| 123/123 [01:51<00:00,  1.18it/s]


Learning rate: 0.0001
Beta 1: 0.95
Beta 2: 0.99
Weight decay: 0.001
Accuracy: 37.5


100%|██████████| 123/123 [01:53<00:00,  1.09it/s]


Learning rate: 0.0001
Beta 1: 0.95
Beta 2: 0.99
Weight decay: 0.0001
Accuracy: 33.48214340209961
Best hyperparameters:
Learning rate: 0.001
Beta 1: 0.8
Beta 2: 0.925
Weight decay: 0.01
Accuracy: 40.625
